# Known Problem:
- **Resolved**
    1. The search "what is a taylor series" returns SSLCertVerification Error. Resolve by adding verify=False to the request.get() function.
        - this resolution does not consider the security of the websites being accessed, potentially use try except to have the request verify when possible and not break when cannot verify



In [7]:
#imports
import requests #send HTTP requests to web pages and retrieve their content
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from urllib.parse import urlparse, urljoin
import pandas as pd
from nltk.corpus import stopwords
import string


In [8]:
import ssl
#ssl._create_default_https_context = ssl._create_unverified_context
#uncomment if you cannot properly download punkt

nltk.download('punkt')
#ssl._create_default_https_context = ssl._create_default_https_context
#uncomment if you cannot properly download punkt

#remove filler words from an example column
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/william/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/william/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
# Step 1: Define the search query and retrieve search results
search_query = input("What would you like to search up?")
search_engine_url = "https://www.google.com/search?q="

# Set headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Number of search result pages to retrieve (each page approx. has 10 results)
num_pages = 2

search_results = []


for page in range(num_pages):
    # Calculate the start index for the current page
    start_index = page * 10

    # Make a GET request to retrieve the search result page
    response = requests.get(search_engine_url + search_query + f"&start={start_index}", headers=headers, verify = False)

    # Step 2: Parse and extract information from the search result page
    soup = BeautifulSoup(response.content, "html.parser")

    for result in soup.select("div.g"):
        title_element = result.select_one("h3")
        if title_element:
            title = title_element.get_text()
            url_element = result.find("a")
            if url_element:
                url = url_element["href"]
                parsed_url = urlparse(url)
                if parsed_url.scheme == "":
                    # Relative URL, convert it to absolute URL
                    base_url = response.url
                    url = urljoin(base_url, url)
                    
            result_response = requests.get(url, headers=headers, verify = False)

            # Extract visible text content from the HTML response
            result_soup = BeautifulSoup(result_response.content, "html.parser")
            text = result_soup.get_text()

            # Remove excess whitespace from the extracted text
            text = ' '.join(text.split())

            search_results.append({"Title": title, "URL": url, "Text": text})

/home/william/anaconda3/envs/search_sense/lib/python3.10/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/william/anaconda3/envs/search_sense/lib/python3.10/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'en.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/william/anaconda3/envs/search_sense/lib/python3.10/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mathworld.wolfram.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.htm

In [36]:
df = pd.DataFrame(search_results)
print(len(df))
#unfiltered dataframe
df.head()

18


,Title,URL,Text
0,Taylor series,https://en.wikipedia.org/wiki/Taylor_series,Taylor series - Wikipedia Jump to content Main...
1,Taylor Series -- from Wolfram MathWorld,https://mathworld.wolfram.com/TaylorSeries.html,Taylor Series -- from Wolfram MathWorld TOPICS...
2,Taylor Series,https://www.mathsisfun.com/algebra/taylor-seri...,Taylor Series Advanced Show Ads Hide Ads About...
3,A Gentle Introduction to Taylor Series,https://machinelearningmastery.com/a-gentle-in...,A Gentle Introduction to Taylor Series - Machi...
4,Calculus II - Taylor Series,https://tutorial.math.lamar.edu/classes/calcii...,Calculus II - Taylor Series Paul's Online Note...


In [37]:
stop_words = stopwords.words("english") #english filler words

def remove_stopwords(text):
    '''
    Input a text block and removes all filler words and special characters, returns a filtered text block in the form of a str
    '''
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = " ".join(filtered_words)
    cleaned_text = filtered_words.translate (str.maketrans('','', string.punctuation))
    return cleaned_text
# def remove_specialchars(text):
#     '''
#     Input a text block and removes all special characters. Returns the filtered text block in the form of a str
#     '''
#     cleaned_text = text.translate (str.maketrans('','', string.punctuation))
#     return cleaned_text

In [38]:
example = df["Text"][0]
filtered_example = remove_stopwords(example)

print("length of unfiltered text: ", len(example))
print("length of filtered text: ", len(filtered_example))

length of unfiltered text:  51507
length of filtered text:  36916


In [39]:
#Properly using the apply() function onto a pandas column
#removes both filler words and special characters
print(len(df["Text"][0]))
df["Text"] = df["Text"].apply(remove_stopwords)

51507


In [40]:
print(len(df["Text"][0]))

36916


In [41]:
#filtered dataframe
df["Text"][0]

'Taylor series  Wikipedia Jump content Main menu Main menu move sidebar hide Navigation Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate Contribute HelpLearn editCommunity portalRecent changesUpload file Languages Language links top page across title Search Search Create accountLog Personal tools Create account Log Pages logged editors learn ContributionsTalk Contents move sidebar hide Top 1Definition 2Examples 3History 4Analytic functions 5Approximation error convergence Toggle Approximation error convergence subsection 51Generalization 6List Maclaurin series common functions Toggle List Maclaurin series common functions subsection 61Exponential function 62Natural logarithm 63Geometric series 64Binomial series 65Trigonometric functions 66Hyperbolic functions 67Polylogarithmic functions 68Elliptic functions 7Calculation Taylor series Toggle Calculation Taylor series subsection 71First example 72Second example 73Third example 8Taylor series definitions 9Taylo